In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
# Import necessary libraries
import os, sys, shutil, glob
import numpy as np
from PIL import Image


def determine_label(label, labels):
	"""
	Definition: Converts label to index in label set

	Parameters: label - label from file
				labels - list of labels
	Returns: index of label in labels (in str format)
	"""
	return str(labels.index(label))

def parse_labels(label_file, labels, img_width, img_height):
	"""
	Definition: Parses label files to extract label and bounding box
		coordinates.  Converts (x1, y1, x1, y2) KITTI format to
		(x, y, width, height) normalized YOLO format.

	Parameters:
	Return: all_labels - contains a list of labels for objects in image
			all_coords - contains a list of coordinate for objects in image
	"""
	lfile = open(label_file)
	coords = []
	all_coords = []
	all_labels = []
	for line in lfile:
		l = line.split(" ")
		all_labels.append(determine_label(l[0], labels))
		coords = list(map(int, map(float, l[4:8])))
		x = float((float(coords[2]) + float(coords[0])) / 2.0) / float(img_width)
		y = float((float(coords[3]) + float(coords[1])) / 2.0) / float(img_height)
		width = float(float(coords[2]) - float(coords[0])) / float(img_width)
		height = float(float(coords[3]) - float(coords[1])) / float(img_height)
		tmp = [x, y, width, height]
		all_coords.append(tmp)
	lfile.close()
	return all_labels, all_coords

def copy_images(path):
	"""
	Definition: Copy all images from the training and validation image sets
		in kitti format to training and validation image sets in yolo format.
		This means converting from .png to .jpg

	Parameters: path - datasets directory
	Returns: None
	"""
	for filename in glob.glob(os.path.join(path + "/training/image_2/", "*.*")):
		shutil.copy(filename, path + "/yolo-train/images/")
	# for filename in glob.glob(os.path.join(path + "val/images-kitti/", "*.*")):
	# 	shutil.copy(filename, path + "val/images-yolo/")

	for filename in glob.glob(os.path.join(path + "/yolo-train/images/", "*.*")):
		im = Image.open(filename)
		im.save(filename.split(".png")[0] + ".jpg", "jpeg")
		os.remove(filename)
	# for filename in glob.glob(os.path.join(path + "val/images-yolo/", "*.*")):
	# 	im = Image.open(filename)
	# 	im.save(filename.split(".png")[0] + ".jpg", "jpeg")
	# 	os.remove(filename)

def write_txt_files(path, f_train):
	"""
	Definition: Fill in a text file containing a list of all images in the
		training and validation image sets.

	Parameters: path - dataset directory
				f_train - file open for adding training examples
				f_val - file open for adding validation examples
	Returns: None
	"""
	for filename in glob.glob(os.path.join(path + "/training/image_2/", "*.*")):
		filename = filename.replace("\\","/")
		f_train.write('%s\n' % (filename))
	# for filename in glob.glob(os.path.join(path + "val/images-yolo/", "*.*")):
	# 	f_val.write('%s\n' % (filename))


def make_yolo_directories(path):
	"""
	Definition: Make directories for yolo images and labels.
		Removes previously created yolo image and label directories.

	Parameters: dataset directory
	Returns: None
	"""
	# if os.path.exists(path + "/yolo-train/images/"):
	# 	shutil.rmtree(path + "/yolo-train/images/")
	if os.path.exists(path + "/training/yolo-labels/"):
		shutil.rmtree(path + "/training/yolo-labels/")
	# if os.path.exists(path + "val/images-yolo/"):
	# 	os.rmdir(path + "val/images-yolo/")
	# if os.path.exists(path + "val/labels-yolo/"):
	# 	os.rmdir(path + "val/labels-yolo/")
	# os.makedirs(path + "/yolo-train/images/")
	os.makedirs(path + "/training/yolo-labels/")
	# os.makedirs(path + "val/images-yolo/")
	# os.makedirs(path + "val/labels-yolo/")

In [3]:
os.listdir('/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single')

['testing', 'training', 'kitti.names']

In [4]:

DIR = '/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single'
LABELS = '/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/kitti.names'

# Split label file
label_file = open(LABELS)
labels_split = label_file.read().split('\n')
print(labels_split)

make_yolo_directories(DIR)

# Go through training data
for f in os.listdir(DIR + "/training/label_2/"):
    fname = (DIR + "/training/image_2/" + f).split(".txt")[0] + ".png"
    print(fname)
    if os.path.isfile(fname):
        img = Image.open(fname)
        w, h = img.size
        labels, coords = parse_labels(os.path.join(DIR + "/training/label_2/" + f),
            labels_split, w, h)
        yolof = open(DIR + "/training/yolo-labels/" + f, "a+")
        for l, c, in zip(labels, coords):
            yolof.write(l + " " + str(c[0]) + " " + str(c[1]) +
                " " + str(c[2]) + " " + str(c[3]) + "\n")
        yolof.close()

f_train = open(DIR + "/yolo-train.txt", "a")
write_txt_files(DIR, f_train)
f_train.close()
print("**COMPLETE***")

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/003603.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/003813.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/005797.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/005615.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/006507.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/003504.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/003885.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/003566.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/006048.png
/content/drive/MyDrive/Colab Notebooks/ML Project/kitti_single/training/image_2/006054.png
/content/drive/MyDrive/Colab Notebooks/